# Remote Unix — Lab 03 

## Preliminary steps

1) Open your terminal
2) Go to any working directory for homeworks
3) Clone the repo: https://github.com/Muxas/fse4ai_2025
4) cd fse4ai_2025
5) Launch test environment: `docker compose up -d fse4ai_04_server fse4ai_04_client`
6) Enter the “client” container: `docker exec -it fse4ai-fse4ai_04_client-1 bash` (name may differ)
7) Remote server: `fse4ai_04_server`
8) Remote user: `student`
9) Remote password: `fse4ai`


In [ ]:
%%bash
git clone https://github.com/Muxas/fse4ai_2025
cd fse4ai_2025
docker compose up -d fse4ai_04_server fse4ai_04_client
docker exec -it fse4ai-fse4ai_04_client-1 bash
echo "server: fse4ai_04_server, user: student, pass: fse4ai"


## 1) ssh-keygen

Generate an ed25519 key (1024 bits option accepted) and save it as files `new_key_ed25519` and `new_key_ed25519.pub`. Then change its passphrase.


In [ ]:
%%bash
ssh-keygen -t ed25519 -b 1024 -f new_key_ed25519
ssh-keygen -p -f new_key_ed25519


## 2) ssh-copy-id

Dry-run adding the new SSH key to `student@fse4ai_04_server`, then actually copy it.


In [ ]:
%%bash
ssh-copy-id -n -i new_key_ed25519.pub student@fse4ai_04_server
ssh-copy-id -i new_key_ed25519.pub student@fse4ai_04_server


## 3) nohup long-running job on remote

a) SSH to the remote with the new key, run `nohup bash -c 'sleep 10; echo complete'` in background, redirect stdout to `std.out` and stderr to `std.err`, then disconnect.
b) Do the same as (a) but in a **single** terminal command (fix: `sleep 10`, not `sleep10`).


In [ ]:
%%bash
ssh -i new_key_ed25519 student@fse4ai_04_server <<'EOF'
nohup bash -c 'sleep 10; echo complete' > std.out 2> std.err &
exit
EOF
ssh -f -i new_key_ed25519 student@fse4ai_04_server 'nohup bash -c "sleep 10; echo complete" > std.out 2> std.err &'


## 4) Redirections, tee, and process substitution (client container)

a) Redirect stdout of `ls -lh` to stderr.
b) Redirect stdout and stderr into separate files `std.out` and `std.err`.
c) Use `tee` so stdout is both saved to `std.out` and shown in terminal; stderr goes to `std.err`.
d) Use process substitution so `ls -lh` stdout is tee'd into `std.out` and stderr tee'd into `std.err`.


In [ ]:
%%bash
ls -lh 1>&2
ls -lh > std.out 2> std.err
ls -lh 2> std.err | tee std.out
ls -lh > >(tee -a std.out) 2> >(tee -a std.err >&2)


## 5) Copying/downloading data with rsync (client container)

a) Copy all files from `~/logs` on the remote into local `logs_local` **only if** size is between 1KB and 1MB; show progress.
b) Copy all files from `~/logs` on the remote into local `logs_local` **only if the filename does not contain `10`**; show progress.


In [ ]:
%%bash
rsync -a --progress -e 'ssh -i new_key_ed25519' --max-size=1M --min-size=1K   student@fse4ai_04_server:logs logs_local

rsync -a --progress -e 'ssh -i new_key_ed25519' --exclude='*10*'   student@fse4ai_04_server:logs logs_local
